# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
! pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
! pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
! pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

Answer: 
1. Efficient indexing: Qdrant uses specialized data structures and indexing techniques like Hierarchical Navigable Small World (HNSW) to enable fast approximate nearest neighbor search
2. Optimized distance metrics: Qdrant supports efficient distance metrics like Cosine Similarity, Dot Product, and Euclidean Distance for measuring vector similarities
3. Collection-based organization: Vectors are organized into collections, allowing for efficient management and search within specific sets of data
4. Flexible vector representations: Qdrant supports named vectors, allowing multiple vectors with different dimensionalities and metrics within a single point, enhancing versatility and performance for complex use cases
5. Memmap storage: As an alternative to in-memory storage, Qdrant provides memmap storage, which creates a virtual address space associated with files on disk, balancing performance and memory usage

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '002df472e220464dad092aa69d49131c', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [46]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Answer:  
The user prompt is passed in via `question` property.  
`context` is retrieved by passing the `question` to the QDrant vector store retriever.  
`RunnablePassthrough.assign` explicitly adds the generated context to the input dict and converts the dicts in the chain to Runnable.  
Noticed `RunnablePassthrough()` works similarly as well.  
The final part captures the response from `primary_qa_llm` for the given `prompt` in `response` and the context in `context`

Let's test it out!

In [47]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of it.


In [48]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '1eb2d2dd2e2441fe9705a887996c9561', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [49]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

Answer:
1. Diverse chunk sizes: Using different chunk sizes (200 and 600 in this case) allows for training and evaluating models on various text lengths, making them more robust and adaptable to different input sizes.  
2. Context preservation: The larger chunk size (600) for evaluation documents helps preserve more context within each chunk for generating synthetic ground truths  
3. Overfitting prevention: Using different parameters for training and evaluation data helps prevent the model from overfitting to a specific chunk size or structure, making it more generalizable  

In [50]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [51]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo") # <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
#critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

/Users/s.thirunavukkarasu/code/AI-Engineering-3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                     
Generating: 100%|██████████| 20/20 [03:38<00:00, 10.91s/it]


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"What is the significance of the concept of ""un...",[that chance is immune from human intervention...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does the Liking/Loving Tendency influence ...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency influences an indiv...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What type of employees are typically hired as ...,[of creating value. And new hires will by deXn...,New hires are typically conservative and chang...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,How can blogging about their point of view hel...,[looking for funding to blog — about their sta...,Blogging about their point of view can help an...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,How can focusing on creating a large market be...,"[developing a large market, as opposed to Xght...",Focusing on creating a large market can be mor...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is a key feature of Peter Hamilton's stor...,"[pandemics, nuclear terrorist attacks, governm...",Peter Hamilton's storytelling style in his spa...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,Why is intelligence considered overrated in th...,[Criteria 7rst\nLots of people will tell you t...,Intelligence is considered overrated in the in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the purpose of temporary subfolders in...,[you can reply to a lot of messages with “I’m ...,The purpose of temporary subfolders in email o...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,Are the anticipated career peaks of highly pro...,[These three components are conspicuously link...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,Is attempting to improve your batting average ...,[becomes irrelevant to determining the success...,"Yes, attempting to improve your batting averag...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Answer:  
This mapping specifies the evolution type of the questions to be generated and their respective proportions within the given test_size. For this mapping, for test_size=100, we expect 50 simple, 25 reasoning and 25 multi_context type questions.

Let's look at the output and see what we can learn about it!

In [75]:
testset.test_data[0]

DataRow(question='What is the significance of the concept of "unwilled" in the context of chance and human interventions?', contexts=['that chance is immune from human interventions. However, one\nmust be careful not to read any unconsciously purposeful intent\ninto “interventions”… [which] are to be viewed as accidental,\nunwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…\nWe can observe chance arriving in four major forms and for four\ndiWerent reasons. The principles involved aWect everyone.\nHere’s where it helps to be a neurologist writing on this topic:\nThe four kinds of chance each have a diWerent kind of motor'], ground_truth='nan', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page':

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [76]:
test_df = testset.to_pandas()

In [77]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"What is the significance of the concept of ""un...",[that chance is immune from human intervention...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does the Liking/Loving Tendency influence ...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency influences an indiv...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What type of employees are typically hired as ...,[of creating value. And new hires will by deXn...,New hires are typically conservative and chang...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,How can blogging about their point of view hel...,[looking for funding to blog — about their sta...,Blogging about their point of view can help an...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,How can focusing on creating a large market be...,"[developing a large market, as opposed to Xght...",Focusing on creating a large market can be mor...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,What is a key feature of Peter Hamilton's stor...,"[pandemics, nuclear terrorist attacks, governm...",Peter Hamilton's storytelling style in his spa...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,Why is intelligence considered overrated in th...,[Criteria 7rst\nLots of people will tell you t...,Intelligence is considered overrated in the in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What is the purpose of temporary subfolders in...,[you can reply to a lot of messages with “I’m ...,The purpose of temporary subfolders in email o...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,Are the anticipated career peaks of highly pro...,[These three components are conspicuously link...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,Is attempting to improve your batting average ...,[becomes irrelevant to determining the success...,"Yes, attempting to improve your batting averag...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [57]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [58]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [59]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [61]:
print(response_dataset[0])

{'question': 'What is the significance of the concept of "unwilled" in the context of chance and human interventions?', 'answer': 'The significance of the concept of "unwilled" in the context of chance and human interventions is that it emphasizes that chance events are unintentional, accidental, and not driven by human intention.', 'contexts': ['unwilled, inadvertent, and unforseeable.\nIndeed, chance plays several distinct roles when humans react cre-\natively with one another and with their environment…', 'must be careful not to read any unconsciously purposeful intent\ninto “interventions”… [which] are to be viewed as accidental,\nunwilled, inadvertent, and unforseeable.', 'ably.\nFirst, he deXnes chance as follows:\nChance… something fortuitous that happens unpredictably without\ndiscernable human intention.\nYup, that’s luck.', 'discernable human intention.\nYup, that’s luck.\nChance is unintentional, it is capricious, but we needn’t conclude\nthat chance is immune from human int

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [62]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [63]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 75/75 [02:22<00:00,  1.90s/it]


In [64]:
results

{'faithfulness': 0.6635, 'answer_relevancy': 0.8130, 'context_recall': 0.4978, 'context_precision': 0.6556, 'answer_correctness': 0.5280}

In [66]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,"What is the significance of the concept of ""un...","The significance of the concept of ""unwilled"" ...","[unwilled, inadvertent, and unforseeable.\nInd...",nan,1.000000,0.983218,0.800000,0.583333,0.182749
1,How does the Liking/Loving Tendency influence ...,The Liking/Loving Tendency influences an indiv...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency influences an indiv...,0.500000,0.957709,0.000000,0.333333,0.618554
2,What type of employees are typically hired as ...,Ambitious director- or VP-level managers.,"[number of new employees recruited and hired, ...",New hires are typically conservative and chang...,0.000000,0.846774,0.000000,0.000000,0.203513
3,How can blogging about their point of view hel...,Blogging about their point of view can help an...,"[esting things going on, about their point of ...",Blogging about their point of view can help an...,0.875000,0.975708,1.000000,1.000000,0.697192
4,How can focusing on creating a large market be...,Focusing on creating a large market can be mor...,"[competitor, be sure to take a step back and s...",Focusing on creating a large market can be mor...,1.000000,0.992700,1.000000,1.000000,0.613491
5,What is a key feature of Peter Hamilton's stor...,Large-scale military combat,[story of an enigmatic agent for the all-power...,Peter Hamilton's storytelling style in his spa...,1.000000,0.836724,1.000000,0.916667,0.192828
6,Why is intelligence considered overrated in th...,Intelligence is considered overrated in the in...,"[Especially in this industry.\nYou will read, ...",Intelligence is considered overrated in the in...,0.500000,1.000000,1.000000,0.333333,0.609262
7,What is the purpose of temporary subfolders in...,The purpose of temporary subfolders in email o...,"[the normal course of your day.\nFourth, aside...",The purpose of temporary subfolders in email o...,0.500000,1.000000,1.000000,0.333333,0.673740
8,Are the anticipated career peaks of highly pro...,Earlier,"[early, end late, and produce at above-average...",nan,NaN,0.000000,0.000000,0.916667,0.953437
9,Is attempting to improve your batting average ...,"No, attempting to improve your batting average...",[improve one’s batting average.\n•\nIntelligen...,"Yes, attempting to improve your batting averag...",0.000000,1.000000,0.000000,0.500000,0.236148


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [78]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [79]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [80]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [81]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift feuding with?"})

In [82]:
print(response["answer"])

I'm sorry, but there is no information provided in the context about Taylor Swift or any feuds she may be involved in.


In [83]:
response = retrieval_chain.invoke({"input": "Why are they feuding?"})

In [84]:
print(response["answer"])

There is no specific mention of a feud in the provided context.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [85]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [86]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [87]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating: 100%|██████████| 75/75 [02:58<00:00,  2.38s/it]


In [88]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,"What is the significance of the concept of ""un...","In the context provided, the concept of ""unwil...","[unwilled, inadvertent, and unforseeable.\nInd...",nan,1.000000,0.926343,1.000000,0.366667,0.179882
1,How does the Liking/Loving Tendency influence ...,The Liking/Loving Tendency influences an indiv...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency influences an indiv...,0.000000,0.945289,1.000000,0.333333,0.493372
2,What type of employees are typically hired as ...,The context does not specify a particular type...,[How to hire the best people you've\never work...,New hires are typically conservative and chang...,1.000000,0.000000,0.000000,0.000000,0.209091
3,How can blogging about their point of view hel...,Blogging about their point of view can help an...,"[esting things going on, about their point of ...",Blogging about their point of view can help an...,1.000000,0.975714,1.000000,1.000000,0.655896
4,How can focusing on creating a large market be...,Focusing on creating a large market can be mor...,[product families or grow market share. This o...,Focusing on creating a large market can be mor...,1.000000,0.975897,0.000000,1.000000,0.387932
5,What is a key feature of Peter Hamilton's stor...,A key feature of Peter Hamilton's storytelling...,[story of an enigmatic agent for the all-power...,Peter Hamilton's storytelling style in his spa...,1.000000,1.000000,0.666667,0.805556,0.841141
6,Why is intelligence considered overrated in th...,Intelligence is considered overrated in the in...,"[Especially in this industry.\nYou will read, ...",Intelligence is considered overrated in the in...,0.600000,0.872869,1.000000,0.333333,0.557292
7,What is the purpose of temporary subfolders in...,The purpose of temporary subfolders in email o...,"[matter, right now.\nThose subfolders then get...",The purpose of temporary subfolders in email o...,0.666667,1.000000,1.000000,0.250000,0.490008
8,Are the anticipated career peaks of highly pro...,The anticipated career peaks of highly product...,[creator’s most distinguished work will appear...,nan,1.000000,0.000000,0.000000,0.833333,0.185286
9,Is attempting to improve your batting average ...,"No, attempting to improve your batting average...",[improve one’s batting average.\n•\nIntelligen...,"Yes, attempting to improve your batting averag...",0.333333,1.000000,1.000000,0.500000,0.228139


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [89]:
results

{'faithfulness': 0.6635, 'answer_relevancy': 0.8130, 'context_recall': 0.4978, 'context_precision': 0.6556, 'answer_correctness': 0.5280}

And see how our advanced retrieval modified our chain!

In [90]:
advanced_retrieval_results

{'faithfulness': 0.6941, 'answer_relevancy': 0.8202, 'context_recall': 0.5778, 'context_precision': 0.6151, 'answer_correctness': 0.4697}

In [91]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.663462,0.694113,0.030651
1,answer_relevancy,0.812962,0.820158,0.007196
2,context_recall,0.497778,0.577778,0.080000
3,context_precision,0.655556,0.615093,-0.040463
4,answer_correctness,0.528001,0.469707,-0.058294


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [93]:
# Init OpenAI text-embedding-3-small model
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [94]:
# Create a in memory Qdrant vector store
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

In [95]:
# Get a retriever for the vector store
new_retriever = vector_store.as_retriever()

In [96]:
# Create an advanced retriever using MQR
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [97]:
# Create a retrieval chain using the new advanced retriever and existing document chain
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [98]:
# Collect answers of all test_questions from the improved chain that uses TE3 + MQR
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [99]:
# Make a dataset from the collected answers and contexts for evaluation
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [101]:
from ragas.run_config import RunConfig

# Evaluate the response data based on the earlier defined metrics. run_config is provided with 5 workers (15 default) to help circumvent TPM rate limit of gpt-3.5-turbo
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics, run_config=RunConfig(max_workers=5))

Evaluating: 100%|██████████| 75/75 [03:05<00:00,  2.47s/it]


In [102]:
new_advanced_retrieval_results

{'faithfulness': 0.8017, 'answer_relevancy': 0.8873, 'context_recall': 0.6444, 'context_precision': 0.6695, 'answer_correctness': 0.5337}

In [103]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.663462,0.694113,0.801667,0.107554,0.138205
1,answer_relevancy,0.812962,0.820158,0.887328,0.067171,0.074366
2,context_recall,0.497778,0.577778,0.644444,0.066667,0.146667
3,context_precision,0.655556,0.615093,0.669511,0.054418,0.013955
4,answer_correctness,0.528001,0.469707,0.533708,0.064000,0.005706


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

Answer:  
Yes. All metrics have positively improved with `text-embedding-3-small` (TE3 + MQR) compared to `ada` (ADA + MQR)

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [104]:
### YOUR CODE HERE
latest_qa_llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

latest_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=latest_qa_llm)

latest_retrieval_chain = create_retrieval_chain(latest_retriever, document_chain)

answers = []
contexts = []

for question in test_questions:
  response = latest_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

latest_response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

latest_results = evaluate(latest_response_dataset, metrics, run_config=RunConfig(max_workers=5))

latest_results

Evaluating: 100%|██████████| 75/75 [03:55<00:00,  3.14s/it]


{'faithfulness': 0.7593, 'answer_relevancy': 0.8751, 'context_recall': 0.6000, 'context_precision': 0.5880, 'answer_correctness': 0.5628}

In [105]:
df_latest = pd.DataFrame(list(latest_results.items()), columns=['Metric', 'GPT4o + TE3 + MQR'])

df_merged = pd.merge(df_merged, df_latest, on="Metric")

df_merged['TE3 + MQR -> GPT4o + TE3 + MQR'] = df_merged['GPT4o + TE3 + MQR'] - df_merged['TE3 + MQR']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR,GPT4o + TE3 + MQR,TE3 + MQR -> GPT4o + TE3 + MQR
0,faithfulness,0.663462,0.694113,0.801667,0.107554,0.138205,0.759259,-0.042407
1,answer_relevancy,0.812962,0.820158,0.887328,0.067171,0.074366,0.875060,-0.012268
2,context_recall,0.497778,0.577778,0.644444,0.066667,0.146667,0.600000,-0.044444
3,context_precision,0.655556,0.615093,0.669511,0.054418,0.013955,0.587955,-0.081556
4,answer_correctness,0.528001,0.469707,0.533708,0.064000,0.005706,0.562839,0.029131


**Hypothesis**

As the ground truths were generated using an older model `gpt-3.5-turbo-16k` and `gpt-4o`'s suspectedly better answers may have scored lower in the RAGAS evaluation metrics.